Install Libraries

In [145]:
import pandas as pd
import numpy as np
import warnings
import panel as pn
pn.extension('tabulator')
import hvplot.pandas

In [147]:
df = pd.read_csv('C:\\Users\\seven\\Downloads\\electricity.csv')
df.head(5)

,Continent,Year,Biofuel Electricity,Coal Electricity,Fossil Fuel Electricity,Gas Electricity,Hydro Electricity,Renewable Electricity,Energy per Capita,CO2 Emmisions Annual
0,Asia,2000,27.54,2106.82,3693.59,931.16,564.26,44.55,9864.972,0.252652
1,Asia,2001,28.79,2248.81,3850.28,1005.01,609.76,45.28,10068.242,0.248706
2,Asia,2002,30.26,2454.56,4139.67,1067.54,634.35,46.97,10431.763,0.249808
3,Asia,2003,34.01,2767.16,4557.87,1163.31,661.85,49.84,11106.729,0.255244
4,Asia,2004,35.90,3020.03,4881.01,1239.53,772.88,52.56,12024.823,0.246679


Create interactive dataframe

In [150]:
idf = df.interactive()

In [152]:
year_slider = pn.widgets.IntSlider(name = 'Generation Year', start = 2000, end = 2022, step = 1, value = 2011)
year_slider

IntSlider(end=2022, name='Generation Year', start=2000, value=2011)

In [154]:
y_axis_electricity = pn.widgets.RadioButtonGroup(name = 'Y Axis', options = ['Hydro Electricity',
                    'Renewable Electricity'], button_type = 'success')
y_axis_electricity

RadioButtonGroup(button_type='success', name='Y Axis', options=['Hydro Electricity', ...], value='Hydro Electricity')

In [156]:
continents = ['Africa', 'Asia', 'Australia', 'Europe', 'North America', 'South America']

Combine Widget with Interactive Dataframe

In [159]:
electricity_pipeline = (idf[(idf.Year <= year_slider) &
                        (idf.Continent.isin(continents))]
                        .groupby(['Continent', 'Year'])[y_axis_electricity].mean()
                        .to_frame()
                        .reset_index()
                        .sort_values(by = 'Year')
                        .reset_index(drop = True))
electricity_pipeline


In [160]:
ect_plot = electricity_pipeline.hvplot.line(x = 'Year', by = 'Continent',
                                           y = y_axis_electricity, line_width =2, cmap = 'Set3',
                                          title = 'Electricity Generation by Continent')
ect_plot

Energy per Capita

In [164]:
y_axis_kwh = pn.widgets.RadioButtonGroup(name = 'Y Axis', options = ['Energy per Capita'], button_type = 'primary')
y_axis_kwh

RadioButtonGroup(button_type='primary', name='Y Axis', options=['Energy per Capita'], value='Energy per Capita')

In [166]:
kwh_pipeline = (idf[(idf.Year <= year_slider) &
                        (idf.Continent.isin(continents))]
                        .groupby(['Continent', 'Year'])[y_axis_kwh].mean()
                        .to_frame()
                        .reset_index()
                        .sort_values(by = 'Year')
                        .reset_index(drop = True))
kwh_pipeline

In [168]:
kwh_plot = kwh_pipeline.hvplot.line(x = 'Year', by = 'Continent',
                                           y = y_axis_kwh, line_width =2, cmap = 'Set3',
                                          title = 'Energy Gerneration per Capita')
kwh_plot

Interactive Table

In [171]:
electricity_table = electricity_pipeline.pipe(pn.widgets.Tabulator, pagination = 'remote',
                    page_size = 10, sizing_mode = 'stretch_width')
electricity_table

Create a multi-variable bar chart.

Displays the comparative dataset of electricity type between the years 2000-2022 for each continent

In [174]:
y_axis_electricity_source = pn.widgets.RadioButtonGroup(name = 'Y Axis', options = ['Biofuel Electricity', 'Coal Electricity', 'Fossil Fuel Electricity',
                                                                                   'Gas Electricity'],
                                                       button_type = 'primary')
y_axis_electricity_source                                                    

RadioButtonGroup(button_type='primary', name='Y Axis', options=['Biofuel Electricity', ...], value='Biofuel Electricity')

In [176]:
continents_world = ['Africa', 'Asia', 'Australia', 'Europe',
                    'North America', 'South America']
electricity_bar_pipeline = (
    idf[(idf.Year <= year_slider) &
    (idf.Continent.isin(continents))]
    .groupby(['Continent', 'Year'])[y_axis_electricity_source].mean()
    .to_frame()
    .reset_index()
    .sort_values(by = 'Year')
    .reset_index(drop = True))
electricity_bar_pipeline

Link the pipeline to dataset to generate the time-series comparative bar chart

In [181]:
electricity_source_bar_plot = electricity_bar_pipeline.hvplot.bar(x= 'Continent',
           y = y_axis_electricity_source, line_width = 2, cmap = 'Set3',
           title = 'Time-Series Electricity \nGeneration Continental Comparison')
electricity_source_bar_plot

Comine all charts into a single dashboard

In [210]:
template = pn.template.FastListTemplate(
    title='Electricity Generation by Continents',
    sidebar=[
        pn.pane.Markdown('# Electricity Generation and Emissions'),
        pn.pane.Markdown('#### Fossil fuels are one of the greatest contributors to pollution via Carbon IV oxide. The increased levels of Carbon IV Oxide result in higher world temperature, hence global warming alters weather patterns leading to climate change. Since fossil fuels are used to generate electricity, it is vital to cut down their usage and switch to alternative electricity sources. Here, we present the amount of electricity generated by different continents from 2000 to 2022'),
        pn.pane.Markdown('## Settings'),
        year_slider
    ],
    main=[
        pn.Row(pn.Column(y_axis_electricity, ect_plot.panel(width=700), margin=(0, 25)),
                electricity_table.panel(width=500)),
        pn.Row(pn.Column(kwh_plot.panel(width=700), margin=(0, 25))),  # Fixed margin definition
        pn.Column(y_axis_electricity_source, electricity_source_bar_plot.panel(width=600))
    ],
    accent_base_color='#88d8b0',
    header_background='#88d8b0',
)

template.servable()

FastListTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] TemplateActions()
    [browser_info] BrowserInfo(dark_mode=False, device_pixel_ratio=1.25, language='en-US', timezone='America/Los_Angeles', timezone_offset=420, webdriver=False)
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [main-2317407420048] Row
        [0] Column(margin=(0, 25))
            [0] RadioButtonGroup(button_type='success', name='Y Axis', options=['Hydro Electricity', ...], value='Hydro Electricity')
            [1] ParamFunction(function, _pane=HoloViews, defer_load=False, width=700)
        [1] ParamFunction(function, _pane=Tabulator, defer_load=False, width=500)
    [main-2317409828688] Row
        [0] Column(margin=(0, 25))
            [0] ParamFunction(function, _pane=HoloViews, defer_load=False, width=700)
    [main-2317409815696] Column
        [0] RadioButtonGroup(button_type='primary', name='Y Axis', options=['Biofuel Electricity', ...], value='Biofuel Electricity')
        [1] ParamFunction(function, _pane=HoloViews, defer_load=False, width=600)
    [nav-2317435688272] Markdown(str)
    [nav-2317436236496] Markdown(str)
    [nav-2317436217808] Markdown(str)
    [nav-2317406993424] IntSlider(end=2022, name='Generation Year', start=2000, value=2011)